# Topic Modeling 
This notebook aims to take as input the texts who have been processed and use it to find the most relevants topics and the words that are relevant for the sentimental analysis.

**Implementation**
- TF-IDF
- FinBERT
- LSA 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import glob
import re
import pandas as pd
import import_ipynb
from pre_processing import processing
from finbert_embedding.embedding import FinbertEmbedding

importing Jupyter notebook from pre_processing.ipynb


[nltk_data] Downloading package stopwords to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ValueError: invalid literal for int() with base 10: 'data/earning_call\\4427353'

### Import the text and process it 

In [ ]:
list_articles = glob.glob("data/earning_call/*")
texts = []
first_sentence = []
articles = []
for s in list_articles:
    with open(s) as f:
        x = int(re.sub('data/earning_call/','',s))
        articles.append(x)
        t = f.read()
        texts.append(t)
        
print('Number of articles', len(texts))

In [ ]:
texts = [processing(x) for x in texts]

### TF-IDF
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(texts)
X_tfidf.shape

In [ ]:
dict_w_index = vectorizer.vocabulary_
dict_index_w = {v: k for k, v in dict_w_index.items()}

Take the top n words depending on the score with the TF-IDF

In [ ]:
n =10
top_n = []
for i in range(X_tfidf.shape[0]):
    index = X_tfidf[i,].nonzero()[1]
    words_of_index = [dict_index_w[x] for x in index]
    score_of_index = [X_tfidf[i,x] for x in index]
    x = list(zip(words_of_index,score_of_index))
    x.sort(key=lambda x: -x[1])
    a = [w[0] for w in x[:n]]
    top_n.append(a)

In [ ]:
df = pd.DataFrame({'article':articles,'file_path':list_articles,'top_n_words':top_n})
df.to_pickle("data/top_n_words_tfidf.pkl")  

In [ ]:
df

### FinBERT 
https://pypi.org/project/finbert-embedding/

In [ ]:
finbert = FinbertEmbedding()

In [ ]:
X_FinB = np.zeros((len(texts),768))
k=0
for text in texts:
    X_FinB[k,] = finbert.sentence_vector(text)
    k+=1

In [ ]:
X_FinB.shape